## Data Cleaning with Python

According to IBM Data Science, you spend more than 80% of your time cleaning data. It is a crucial stage in Data Science and Analysis. 

We will work on cleaning data using pandas library and work on the biggest data cleaning task, "Handling missing values"

Some source of missing data could include 
- User forgot to fill in a field.
- Data was lost while transferring manually from a legacy database.
- There was a programming error.
- Users chose not to fill out a field tied to their beliefs about how the results would be used or interpreted

### Getting Started

Before you start cleaning a data set, it’s a good idea to just get a general feel for the data. After that, you can put together a plan to clean the data.

We will start by asking the following questions:

- What are the features?
- What are the expected types (int, float, string, boolean)?
- Is there obvious missing data (values that Pandas can detect)?
- Is there other types of missing data that’s not so obvious (can’t easily detect with Pandas)?
- Are there duplicates?
- Are the spellings correct or in the right order

**This is a screenshot of what our dataset looks like**

![Picture](realestatepractice_dataset.png)

### Let us load the dataset

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('realestate_dataset.csv')

In [3]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3,Y
1,197.0,LEXINGTON,3,N
2,NaN,LEXINGTON,NaN,N
3,201.0,BERKELEY,1,12
4,203.0,BERKELEY,3,Y
5,207.0,BERKELEY,NaN,Y
6,NaN,WASHINGTON,2,NaN
7,207.0,BERKELEY,NaN,Y
8,213.0,TREMONT,__,Y
9,215.0,TREMONT,na,Y


What are each features

- ST_NUM: Street number
- ST_NAME: Street name
- OWN_OCCUPIED: Is the residence owner occupied
- NUM_BEDROOMS: Number of bedrooms

### Exploring the dataset

In [4]:
# checking some basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ST_NUM        8 non-null      float64
 1   ST_NAME       10 non-null     object 
 2   NUM_BEDROOMS  7 non-null      object 
 3   OWN_OCCUPIED  9 non-null      object 
dtypes: float64(1), object(3)
memory usage: 448.0+ bytes


In [5]:
# check how many null values in each column
df.isnull().sum()

ST_NUM          2
ST_NAME         0
NUM_BEDROOMS    3
OWN_OCCUPIED    1
dtype: int64

In [6]:
# to check only the columns with missing values
missing = df.isnull().sum()
missing = missing[missing > 0]
missing

ST_NUM          2
NUM_BEDROOMS    3
OWN_OCCUPIED    1
dtype: int64

In [7]:
# find unique values of each column
for col in df.columns:
    values = df[col].unique()
    print(f'{col}: {values}')
    

ST_NUM: [104. 197.  nan 201. 203. 207. 213. 215.]
ST_NAME: ['PUTNAM' 'LEXINGTON' 'BERKELEY' 'WASHINGTON' 'TREMONT']
NUM_BEDROOMS: ['3' nan '1' '2' '__' 'na']
OWN_OCCUPIED: ['Y' 'N' '12' nan]


## 1. Detecting Missing Values

### Standard Missing Values

- These are missing values that pandas can detect

In [8]:
print(df['ST_NUM'])
print(df['ST_NUM'].isnull())

0    104.0
1    197.0
2      NaN
3    201.0
4    203.0
5    207.0
6      NaN
7    207.0
8    213.0
9    215.0
Name: ST_NUM, dtype: float64
0    False
1    False
2     True
3    False
4    False
5    False
6     True
7    False
8    False
9    False
Name: ST_NUM, dtype: bool


**Pandas recognises an empty space and NA as missing values**

### Non-Standard Missing Values

- These are missing values that pandas cannot detect

In [9]:
print(df['NUM_BEDROOMS'])
print(df['NUM_BEDROOMS'].isnull())

0      3
1      3
2    NaN
3      1
4      3
5    NaN
6      2
7    NaN
8     __
9     na
Name: NUM_BEDROOMS, dtype: object
0    False
1    False
2     True
3    False
4    False
5     True
6    False
7     True
8    False
9    False
Name: NUM_BEDROOMS, dtype: bool


**Pandas recognizes n/a and NA as missing values**

**Pandas does not recognize __ and na as missing values**

A way to make pandas catch a missing value happens before loading. Make a list of all possible missing value format and put in a list, then call it during loading.

In [10]:
missing_values = ['NA', 'N/A', '__', 'na', 'n/a']
df = pd.read_csv('realestate_dataset.csv', na_values = missing_values)

In [11]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,NaN,LEXINGTON,NaN,N
3,201.0,BERKELEY,1.0,12
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,NaN,Y
6,NaN,WASHINGTON,2.0,NaN
7,207.0,BERKELEY,NaN,Y
8,213.0,TREMONT,NaN,Y
9,215.0,TREMONT,NaN,Y


### Unexpected Missing Values

- For example, if our feature is expected to be a string, but there is a numeric type, then technically it is also a missing value. 

In [12]:
df[['OWN_OCCUPIED']]

,OWN_OCCUPIED
0,Y
1,N
2,N
3,12
4,Y
5,Y
6,NaN
7,Y
8,Y
9,Y


Own occupied column should either show Y or N, meaning yes or no, but we are seeing a 12. What is 12??? That in itself is a missing value

In [13]:
df1 = df.copy()

In [14]:
df1

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,NaN,LEXINGTON,NaN,N
3,201.0,BERKELEY,1.0,12
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,NaN,Y
6,NaN,WASHINGTON,2.0,NaN
7,207.0,BERKELEY,NaN,Y
8,213.0,TREMONT,NaN,Y
9,215.0,TREMONT,NaN,Y


In [15]:
df1.loc[3, 'OWN_OCCUPIED'] = np.nan

In [16]:
df1

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,NaN,LEXINGTON,NaN,N
3,201.0,BERKELEY,1.0,NaN
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,NaN,Y
6,NaN,WASHINGTON,2.0,NaN
7,207.0,BERKELEY,NaN,Y
8,213.0,TREMONT,NaN,Y
9,215.0,TREMONT,NaN,Y


In [17]:
row_num = 0
for row in df['OWN_OCCUPIED']:
    try:
        int(row)
        df.loc[row_num, 'OWN_OCCUPIED'] = np.nan
    except ValueError:
        pass
    row_num += 1


In [18]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,NaN,LEXINGTON,NaN,N
3,201.0,BERKELEY,1.0,NaN
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,NaN,Y
6,NaN,WASHINGTON,2.0,NaN
7,207.0,BERKELEY,NaN,Y
8,213.0,TREMONT,NaN,Y
9,215.0,TREMONT,NaN,Y


___

## 2. Handling the missing values

Now all the missing values are showing as NaN in our dataset, it is time to handle them.

In [19]:
df.isnull().sum()

ST_NUM          2
ST_NAME         0
NUM_BEDROOMS    5
OWN_OCCUPIED    2
dtype: int64

### 2.1. We might handle by replacing with a single value.

In [20]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,NaN,LEXINGTON,NaN,N
3,201.0,BERKELEY,1.0,NaN
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,NaN,Y
6,NaN,WASHINGTON,2.0,NaN
7,207.0,BERKELEY,NaN,Y
8,213.0,TREMONT,NaN,Y
9,215.0,TREMONT,NaN,Y


In [21]:
df['ST_NUM'] = df['ST_NUM'].fillna(125)

#df['ST_NUM'].fillna(125, inplace = True)

In [22]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,125.0,LEXINGTON,NaN,N
3,201.0,BERKELEY,1.0,NaN
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,NaN,Y
6,125.0,WASHINGTON,2.0,NaN
7,207.0,BERKELEY,NaN,Y
8,213.0,TREMONT,NaN,Y
9,215.0,TREMONT,NaN,Y


### 2.2 We might handle by replacing with a mean, median or mode of that column.

A very common way is by using the median

In [23]:
Median = df['NUM_BEDROOMS'].median()
Median

3.0

In [24]:
df['NUM_BEDROOMS'].fillna(Median, inplace = True)


In [25]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,125.0,LEXINGTON,3.0,N
3,201.0,BERKELEY,1.0,NaN
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,3.0,Y
6,125.0,WASHINGTON,2.0,NaN
7,207.0,BERKELEY,3.0,Y
8,213.0,TREMONT,3.0,Y
9,215.0,TREMONT,3.0,Y


### 2.3 We might handle by replacing with different values.


In [26]:
df.loc[3, 'OWN_OCCUPIED'] = "Y"
df.loc[6, 'OWN_OCCUPIED'] = "N"

In [27]:
df[['OWN_OCCUPIED']]

,OWN_OCCUPIED
0,Y
1,N
2,N
3,Y
4,Y
5,Y
6,N
7,Y
8,Y
9,Y


In [28]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,125.0,LEXINGTON,3.0,N
3,201.0,BERKELEY,1.0,Y
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,3.0,Y
6,125.0,WASHINGTON,2.0,N
7,207.0,BERKELEY,3.0,Y
8,213.0,TREMONT,3.0,Y
9,215.0,TREMONT,3.0,Y


___

## 3. Check for duplicates.


In [29]:
df[df.duplicated()]

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
7,207.0,BERKELEY,3.0,Y


In [30]:
# drop duplicates to avoid redundancy

df = df.drop(df[df.duplicated()].index)

In [31]:
df = df.drop(df[df.duplicated()].index)

In [32]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,125.0,LEXINGTON,3.0,N
3,201.0,BERKELEY,1.0,Y
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,3.0,Y
6,125.0,WASHINGTON,2.0,N
8,213.0,TREMONT,3.0,Y
9,215.0,TREMONT,3.0,Y


In [33]:
df = df.reset_index(drop = True)

In [34]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,125.0,LEXINGTON,3.0,N
3,201.0,BERKELEY,1.0,Y
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,3.0,Y
6,125.0,WASHINGTON,2.0,N
7,213.0,TREMONT,3.0,Y
8,215.0,TREMONT,3.0,Y


In [35]:

#axis = 0 row
#axis = 1 column

In [36]:
df2 = df.copy()
df2

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,125.0,LEXINGTON,3.0,N
3,201.0,BERKELEY,1.0,Y
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,3.0,Y
6,125.0,WASHINGTON,2.0,N
7,213.0,TREMONT,3.0,Y
8,215.0,TREMONT,3.0,Y


In [37]:
df2 = df2.drop(5, axis =0)
df2

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,125.0,LEXINGTON,3.0,N
3,201.0,BERKELEY,1.0,Y
4,203.0,BERKELEY,3.0,Y
6,125.0,WASHINGTON,2.0,N
7,213.0,TREMONT,3.0,Y
8,215.0,TREMONT,3.0,Y


In [38]:
df2 = df2.drop('ST_NAME', axis = 1)

In [39]:
df2

,ST_NUM,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,3.0,Y
1,197.0,3.0,N
2,125.0,3.0,N
3,201.0,1.0,Y
4,203.0,3.0,Y
6,125.0,2.0,N
7,213.0,3.0,Y
8,215.0,3.0,Y


___

## 4. Ensure accurate datatype


In [40]:
df.dtypes

ST_NUM          float64
ST_NAME          object
NUM_BEDROOMS    float64
OWN_OCCUPIED     object
dtype: object

In [41]:
df

,ST_NUM,ST_NAME,NUM_BEDROOMS,OWN_OCCUPIED
0,104.0,PUTNAM,3.0,Y
1,197.0,LEXINGTON,3.0,N
2,125.0,LEXINGTON,3.0,N
3,201.0,BERKELEY,1.0,Y
4,203.0,BERKELEY,3.0,Y
5,207.0,BERKELEY,3.0,Y
6,125.0,WASHINGTON,2.0,N
7,213.0,TREMONT,3.0,Y
8,215.0,TREMONT,3.0,Y
